In [1]:
#from models import *
import torch
import pytorchvideo
from pytorchvideo.data import labeled_video_dataset
from pytorchvideo.data.clip_sampling import make_clip_sampler
from pytorchvideo.transforms import ApplyTransformToKey, RandomResizedCrop, Normalize, RandomShortSideScale, RemoveKey, ShortSideScale, UniformTemporalSubsample
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader, ConcatDataset
from tqdm import tqdm
import pytorchvideo.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models.video as v_model
import numpy as np
import matplotlib.pyplot as plt
import os
#from models import *


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
leg = 'R'

In [2]:
train_transform = transforms.Compose(
            [
            ApplyTransformToKey(
              key="video",
              transform = transforms.Compose(
                  [
                    #transforms.RandomRotation(20),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                    transforms.Resize((192,108)),
                  ]
                ),
              ),
            ]
        )
add_transform = transforms.Compose(
            [
            ApplyTransformToKey(
              key="video",
              transform = transforms.Compose(
                  [
                    
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                    transforms.Resize((192,108)),
                    transforms.RandomRotation(20),
                  ]
                ),
              ),
            ]
        )

batch_size = 3
num_sec = 5

In [3]:
train_sampler = make_clip_sampler('uniform', num_sec)
#valid_sampler = make_clip_sampler('uniform', num_sec)
dataset = labeled_video_dataset(f'video_dataset_{leg}', train_sampler, transform=train_transform)
add_dataset = labeled_video_dataset(f'video_dataset_{leg}', train_sampler, transform=add_transform)

#train_set = ConcatDataset([dataset, add_dataset])

valid_set = labeled_video_dataset(f'valid_set_{leg}', train_sampler, transform=train_transform)
#train_set, valid_set = random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])
train_loader = DataLoader(dataset, batch_size = 3)
add_loader = DataLoader(add_dataset, batch_size = 3)

valid_loader = DataLoader(valid_set, batch_size = 3)

In [4]:
model = v_model.r3d_18(weights = 'R3D_18_Weights.DEFAULT')
model = model.to(device)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
sched = lr_scheduler.MultiStepLR(optimizer, milestones=[20,40], gamma=0.1)

#batch_loss = 0

if not os.path.isdir(f'hint_model_{leg}'):
    os.mkdir(f'hint_model_{leg}')

best_loss = np.Inf
epoch_mem = 0

epochs = 50
train_mem, valid_mem = [], []

for epoch in range(epochs):

    train_loss = 0
    accuracy = 0
    total = 0
    
    # start training
    model.train()
    
    # read training data
    for batch in tqdm(train_loader):
        imgs, labels = batch['video'], batch['label']
        imgs, labels = imgs.to(device), labels.to(device)

        # feed the group of images into model and get probability
        outputs = model(imgs)
        
        # make a prediction
        _, preds = torch.max(outputs, 1)
        
        # calculate loss by CrossEntropy
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        
        # update model weight
        loss.backward()
        optimizer.step()
        sched.step()

        # add training loss
        train_loss += loss.item()
        total += imgs.shape[0]
        
        # add training accuracy
        accuracy += torch.sum(preds == labels.data)
        
        # release memory space to prevent out of memory
        torch.cuda.empty_cache()
     
    for batch in tqdm(add_loader):
        imgs, labels = batch['video'], batch['label']
        imgs, labels = imgs.to(device), labels.to(device)

        # feed the group of images into model and get probability
        outputs = model(imgs)
        
        # make a prediction
        _, preds = torch.max(outputs, 1)
        
        # calculate loss by CrossEntropy
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        
        # update model weight
        loss.backward()
        optimizer.step()
        sched.step()

        # add training loss
        train_loss += loss.item()
        total += imgs.shape[0]
        
        # add training accuracy
        accuracy += torch.sum(preds == labels.data)
        
        # release memory space to prevent out of memory
        torch.cuda.empty_cache()
    print(f'Epoch {epoch} | train_accuracy = {accuracy / total:03f}, train_loss = {train_loss / total:03f}')
    train_mem.append(train_loss / total)
    
    valid_loss = 0
    total = 0
    accuracy = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            imgs, labels = batch['video'], batch['label']
            imgs, labels = imgs.to(device), labels.to(device)

            # feed the group of images into model and get probability
            outputs = model(imgs)

            # make a prediction
            _, preds = torch.max(outputs, 1)

            # calculate loss by CrossEntropy
            loss = criterion(outputs, labels)

            optimizer.zero_grad()

            # add training loss
            valid_loss += loss.item()
            total += imgs.shape[0]

            # add training accuracy
            accuracy += torch.sum(preds == labels.data)

            # release memory space to prevent out of memory
            torch.cuda.empty_cache()
    if valid_loss / total < best_loss:
        best_loss = valid_loss / total
        epoch_mem = epoch
        torch.save(model.state_dict(), f'./hint_model_{leg}/right_model_{epoch}.pt' if leg == 'R' else f'./hint_model_{leg}/left_model_{epoch}.pt')
    valid_mem.append(valid_loss / total)
    print(f'Epoch {epoch} | valid_accuracy = {accuracy / total:03f}, valid_loss = {valid_loss / total:03f}')

59it [02:06,  2.15s/it]
59it [02:05,  2.12s/it]


Epoch 0 | train_accuracy = 0.560000, train_loss = 1.080941


23it [00:36,  1.59s/it]


Epoch 0 | valid_accuracy = 0.507246, valid_loss = 0.551438


59it [02:01,  2.06s/it]
59it [02:05,  2.13s/it]


...

Epoch 48 | valid_accuracy = 0.478261, valid_loss = 0.667662


59it [02:09,  2.20s/it]
59it [02:13,  2.27s/it]


Epoch 49 | train_accuracy = 0.991429, train_loss = 0.043381


23it [00:40,  1.75s/it]

Epoch 49 | valid_accuracy = 0.434783, valid_loss = 0.605077


In [6]:
n_epochs = np.arange(epochs)
plt.figure(0)
plt.plot(n_epochs, train_mem, color = 'b', label = 'training loss')
plt.plot(n_epochs, valid_mem, color = 'r', label = 'Evaluation loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
#plt.show()
plt.savefig(f'./hint_model_{leg}/{leg}_loss_surface.png')
plt.clf()

<Figure size 432x288 with 0 Axes>